In [1]:
#import necessary packages
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from sklearn.model_selection import train_test_split
import time
import matplotlib.pyplot as plt

In [26]:
#import data
PL20_21_fixtures = pd.read_csv('Data/2020-21/Fixtures/PL20_21_features.csv')
PL20_21_lineups = pd.read_csv('Data/2020-21/Players/PL2020_lineups.csv')
PL20_21_player_stats = pd.read_csv('Data/2020-21/Players/PL2020_total.csv')
PL20_21_lineups['player_id'] = PL20_21_lineups['player'] + PL20_21_lineups['team'] + PL20_21_lineups['date'].astype(str)
PL20_21_lineups['game_id'] = PL20_21_lineups['team'] + PL20_21_lineups['date'].astype(str)
PL20_21_lineups
PL20_21_player_stats = PL20_21_player_stats.rename(columns={'player_ID':'player_id','game_ID':'game_id'})

In [27]:
PL20_21_player_stats

,Unnamed: 0,player,date,team,matchweek,category,position,minutes,rating,xP,...,creativity,threat,total_points,transfers_balance,transfers_in,transfers_out,value,was_home,player_id,game_id
0,0,Joe Bryan,2020-09-12,Fulham,1.0,Defender,LB,90.0,30.040214,1.1,...,4.3,0.0,1,0,0,0,50,True,Joe BryanFulham2020-09-12,Fulham2020-09-12
1,1,Tim Ream,2020-09-12,Fulham,1.0,Defender,CB,90.0,33.442623,0.6,...,0.8,0.0,1,0,0,0,45,True,Tim ReamFulham2020-09-12,Fulham2020-09-12
2,2,Michael Hector,2020-09-12,Fulham,1.0,Defender,CB,90.0,37.665180,0.6,...,0.6,0.0,0,0,0,0,45,True,Michael HectorFulham2020-09-12,Fulham2020-09-12
3,3,Denis Odoi,2020-09-12,Fulham,1.0,Defender,RB,90.0,33.911034,0.6,...,2.6,7.0,1,0,0,0,45,True,Denis OdoiFulham2020-09-12,Fulham2020-09-12
4,4,Ainsley Maitland-Niles,2020-09-12,Arsenal,1.0,Defender,WB,90.0,25.875207,3.1,...,2.8,4.0,6,0,0,0,50,False,Ainsley Maitland-NilesArsenal2020-09-12,Arsenal2020-09-12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10392,10392,Emiliano Martínez,2021-05-23,Aston Villa,38.0,NaN,GK,90.0,63.001016,2.5,...,0.0,0.0,3,-62612,15191,77803,53,True,Emiliano MartínezAston Villa2021-05-23,Aston Villa2021-05-23
10393,10393,Edouard Mendy,2021-05-23,Chelsea,38.0,NaN,GK,45.0,33.585029,1.5,...,0.0,0.0,1,41035,58884,17849,53,False,Edouard MendyChelsea2021-05-23,Chelsea2021-05-23
10394,10394,Kepa Arrizabalaga,2021-05-23,Chelsea,38.0,NaN,GK,45.0,26.766151,0.5,...,0.0,0.0,0,416,951,535,47,False,Kepa ArrizabalagaChelsea2021-05-23,Chelsea2021-05-23
10395,10395,Łukasz Fabiański,2021-05-23,West Ham United,38.0,NaN,GK,90.0,71.562860,7.1,...,0.0,0.0,9,-25881,4342,30223,50,True,Łukasz FabiańskiWest Ham United2021-05-23,West Ham United2021-05-23


In [28]:
PL20_21_fixtures

,Unnamed: 0,Div,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,...,Elo_h_after,Elo_a_after,Elo_h_before,Elo_a_before,goalsDifference,shotsDifference,cornersDifference,ewmDifference,elo_difference,form_difference
0,0,E0,2020-09-12,12:30,Fulham,Arsenal,0,3,A,0,...,1485.00,1515.00,1500.00,1500.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
1,1,E0,2020-09-12,15:00,Crystal Palace,Southampton,1,0,H,1,...,1515.00,1485.00,1500.00,1500.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
2,2,E0,2020-09-12,17:30,Liverpool,Leeds United,4,3,H,3,...,1515.00,1485.00,1500.00,1500.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
3,3,E0,2020-09-12,20:00,West Ham United,Newcastle United,0,2,A,0,...,1485.00,1515.00,1500.00,1500.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
4,4,E0,2020-09-13,14:00,West Bromwich Albion,Leicester City,0,3,A,0,...,1485.00,1515.00,1500.00,1500.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,375,E0,2021-05-23,16:00,Liverpool,Crystal Palace,2,0,H,1,...,1591.89,1439.73,1582.38,1449.24,2.500000,16.500000,12.833333,19.313782,133.14,5.500000
376,376,E0,2021-05-23,16:00,Manchester City,Everton,5,0,H,2,...,1659.36,1508.70,1649.79,1518.27,2.166667,1.666667,6.000000,24.719180,131.52,1.666667
377,377,E0,2021-05-23,16:00,Sheffield United,Burnley,1,0,H,1,...,1358.16,1420.86,1338.90,1440.12,-4.833333,-8.333333,-12.500000,-19.400129,-101.22,-0.500000
378,378,E0,2021-05-23,16:00,West Ham United,Southampton,3,0,H,2,...,1560.00,1420.08,1549.98,1430.10,2.833333,-3.500000,-6.666667,18.155503,119.88,2.500000


In [29]:
def get_wk(fixtures):
    home = fixtures[['HomeTeam', 'Wk_h', 'game_home_ID']]
    away = fixtures[['AwayTeam', 'Wk_a', 'game_away_ID']]
    home = home.rename(columns={'HomeTeam':'Team','Wk_h':'wk', 'game_home_ID': 'game_id'})
    away = away.rename(columns={'AwayTeam':'Team','Wk_a':'wk', 'game_away_ID': 'game_id'})
    return home.append(away)
game_weeks = get_wk(PL20_21_fixtures)


C:\Users\giuli\AppData\Local\Temp\ipykernel_4408\2442093579.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  return home.append(away)


In [44]:
PL20_21_lineups_tmp =  pd.merge(PL20_21_lineups, game_weeks[['game_id', 'wk']], on= 'game_id', how='left')
PL20_21_player_stats_tmp =  pd.merge(PL20_21_player_stats, game_weeks[['game_id', 'wk']], on= 'game_id', how='left')

In [31]:
def get_lineup(lineups_df, game_id):
    rslt_df = lineups_df[lineups_df['game_id'] == game_id]
    return rslt_df

get_lineup( PL20_21_lineups_tmp, 'Arsenal2020-09-12',)

,player,team,matchweek,date,start11,player_id,game_id,wk
18,Bernd Leno,Arsenal,1.0,2020-09-12,True,Bernd LenoArsenal2020-09-12,Arsenal2020-09-12,1
19,Héctor Bellerín,Arsenal,1.0,2020-09-12,True,Héctor BellerínArsenal2020-09-12,Arsenal2020-09-12,1
20,Kieran Tierney,Arsenal,1.0,2020-09-12,True,Kieran TierneyArsenal2020-09-12,Arsenal2020-09-12,1
21,Gabriel Dos Santos,Arsenal,1.0,2020-09-12,True,Gabriel Dos SantosArsenal2020-09-12,Arsenal2020-09-12,1
22,Alexandre Lacazette,Arsenal,1.0,2020-09-12,True,Alexandre LacazetteArsenal2020-09-12,Arsenal2020-09-12,1
23,Willian,Arsenal,1.0,2020-09-12,True,WillianArsenal2020-09-12,Arsenal2020-09-12,1
24,Pierre-Emerick Aubameyang,Arsenal,1.0,2020-09-12,True,Pierre-Emerick AubameyangArsenal2020-09-12,Arsenal2020-09-12,1
25,Ainsley Maitland-Niles,Arsenal,1.0,2020-09-12,True,Ainsley Maitland-NilesArsenal2020-09-12,Arsenal2020-09-12,1
26,Rob Holding,Arsenal,1.0,2020-09-12,True,Rob HoldingArsenal2020-09-12,Arsenal2020-09-12,1
27,Mohamed Elneny,Arsenal,1.0,2020-09-12,True,Mohamed ElnenyArsenal2020-09-12,Arsenal2020-09-12,1


In [47]:
def get_player_stats(player_stats, lineup, wk):
    tmp = pd.merge(player_stats, lineup[['player', 'start11']], on= 'player')
    df = tmp.drop(tmp[tmp.wk > wk].index)
    return df
lineup = get_lineup(PL20_21_lineups_tmp, 'Arsenal2020-09-12')
get_player_stats(PL20_21_player_stats_tmp, lineup,4)

,Unnamed: 0,player,date,team,matchweek,category,position,minutes,rating,xP,...,total_points,transfers_balance,transfers_in,transfers_out,value,was_home,player_id,game_id,wk,start11
0,4,Ainsley Maitland-Niles,2020-09-12,Arsenal,1.0,Defender,WB,90.0,25.875207,3.1,...,6,0,0,0,50,False,Ainsley Maitland-NilesArsenal2020-09-12,Arsenal2020-09-12,1,True
1,259,Ainsley Maitland-Niles,2020-09-28,Arsenal,3.0,Defender,WB,90.0,28.022187,1.8,...,1,-30409,5689,36098,50,False,Ainsley Maitland-NilesArsenal2020-09-28,Arsenal2020-09-28,3,True
7,3878,Ainsley Maitland-Niles,2020-10-04,Arsenal,4.0,Midfield,CM,4.0,0.923805,2.5,...,1,-9240,4867,14107,50,True,Ainsley Maitland-NilesArsenal2020-10-04,Arsenal2020-10-04,4,True
26,5,Héctor Bellerín,2020-09-12,Arsenal,1.0,Defender,WB,90.0,51.847318,3.1,...,5,0,0,0,50,False,Héctor BellerínArsenal2020-09-12,Arsenal2020-09-12,1,True
27,101,Héctor Bellerín,2020-09-19,Arsenal,2.0,Defender,WB,90.0,22.777974,4.0,...,2,23037,54450,31413,50,True,Héctor BellerínArsenal2020-09-19,Arsenal2020-09-19,2,True
28,260,Héctor Bellerín,2020-09-28,Arsenal,3.0,Defender,WB,90.0,27.926968,1.8,...,0,-19029,39377,58406,50,False,Héctor BellerínArsenal2020-09-28,Arsenal2020-09-28,3,True
29,323,Héctor Bellerín,2020-10-04,Arsenal,4.0,Defender,RB,90.0,40.471093,5.0,...,11,-9896,23203,33099,50,True,Héctor BellerínArsenal2020-10-04,Arsenal2020-10-04,4,True
51,6,Kieran Tierney,2020-09-12,Arsenal,1.0,Defender,CB,90.0,42.918335,3.5,...,6,0,0,0,55,False,Kieran TierneyArsenal2020-09-12,Arsenal2020-09-12,1,True
52,261,Kieran Tierney,2020-09-28,Arsenal,3.0,Defender,CB,90.0,31.343993,1.3,...,1,-248129,7001,255130,55,False,Kieran TierneyArsenal2020-09-28,Arsenal2020-09-28,3,True
53,320,Kieran Tierney,2020-10-04,Arsenal,4.0,Defender,LB,90.0,54.619466,2.7,...,2,-46465,10701,57166,54,True,Kieran TierneyArsenal2020-10-04,Arsenal2020-10-04,4,True


In [50]:
test =  pd.merge(PL20_21_lineups_tmp, PL20_21_player_stats_tmp[['player_id', 'category', 'position', 'minutes', 'rating', 'xP', 'bonus', 'bps', 'ict_index', 'influence', 'creativity', 'threat', 'total_points', 'transfers_in', 'transfers_out', 'transfers_balance', 'value', 'was_home']], on= 'player_id', how='left')
test

,player,team,matchweek,date,start11,player_id,game_id,wk,category,position,...,ict_index,influence,creativity,threat,total_points,transfers_in,transfers_out,transfers_balance,value,was_home
0,Marek Rodák,Fulham,1.0,2020-09-12,True,Marek RodákFulham2020-09-12,Fulham2020-09-12,1,NaN,GK,...,1.8,18.2,0.0,0.0,1.0,0.0,0.0,0.0,45.0,True
1,Michael Hector,Fulham,1.0,2020-09-12,True,Michael HectorFulham2020-09-12,Fulham2020-09-12,1,Defender,CB,...,1.7,16.8,0.6,0.0,0.0,0.0,0.0,0.0,45.0,True
2,Denis Odoi,Fulham,1.0,2020-09-12,True,Denis OdoiFulham2020-09-12,Fulham2020-09-12,1,Defender,RB,...,2.1,11.4,2.6,7.0,1.0,0.0,0.0,0.0,45.0,True
3,Neeskens Kebano,Fulham,1.0,2020-09-12,True,Neeskens KebanoFulham2020-09-12,Fulham2020-09-12,1,Forward,RW,...,2.2,1.0,2.7,18.0,2.0,0.0,0.0,0.0,50.0,True
4,Tom Cairney,Fulham,1.0,2020-09-12,True,Tom CairneyFulham2020-09-12,Fulham2020-09-12,1,Midfield,DM,...,1.7,7.0,10.2,0.0,1.0,0.0,0.0,0.0,55.0,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14641,Michael Obafemi,Southampton,38.0,2021-05-23,False,Michael ObafemiSouthampton2021-05-23,Southampton2021-05-23,38,Forward,FW,...,0.0,0.0,0.3,0.0,1.0,618.0,520.0,98.0,46.0,False
14642,Nathan Tella,Southampton,38.0,2021-05-23,False,Nathan TellaSouthampton2021-05-23,Southampton2021-05-23,38,Forward,FW,...,2.5,4.6,1.4,19.0,1.0,1899.0,1061.0,838.0,43.0,False
14643,Theo Walcott,Southampton,38.0,2021-05-23,False,Theo WalcottSouthampton2021-05-23,Southampton2021-05-23,38,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14644,Dan Nlundulu,Southampton,38.0,2021-05-23,False,Dan NlunduluSouthampton2021-05-23,Southampton2021-05-23,38,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
